#Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with_mask= '/content/drive/MyDrive/Machine Learning/Tubes/Dataset/with_mask'
without_mask = '/content/drive/MyDrive/Machine Learning/Tubes/Dataset/without_mask'

In [ ]:
import os

In [ ]:
print("jumlah masker   :", len(os.listdir(with_mask)))
print("jumlah tanpa masker    :", len(os.listdir(without_mask)))

jumlah masker   : 2165
jumlah tanpa masker    : 1930


#Splitting Dataset

In [ ]:
train_dir = '/content/drive/MyDrive/Machine Learning/Tubes/Splited Dataset/train'
val_dir = '/content/drive/MyDrive/Machine Learning/Tubes/Splited Dataset/val'
test_dir = '/content/drive/MyDrive/Machine Learning/Tubes/Splited Dataset/test'

In [ ]:
import matplotlib.pyplot as plt 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

In [ ]:
batch_size = 20

#Augmentasi Data

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.5,
    fill_mode='constant',
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,150),
    batch_size = batch_size,
    shuffle=True,
    color_mode="rgb",
    class_mode='categorical'
)

val_gen = train_datagen.flow_from_directory(
    val_dir,
    target_size=(100,150),
    batch_size = batch_size,
    color_mode="rgb",
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,150),
    batch_size = batch_size,
    color_mode="rgb",
    class_mode='categorical'
)

Found 3273 images belonging to 2 classes.
Found 777 images belonging to 2 classes.
Found 42 images belonging to 2 classes.


#Set Callback

In [ ]:
#DESIRED_ACCURACY = 0.975

#class myCallback(tf.keras.callbacks.Callback):
#    def on_epoch_end(self, epochs, logs={}) :
#        if(logs.get('val_acc') is not None and logs.get('val_acc') >= DESIRED_ACCURACY) :
#            print('\nReached 97,5% accuracy so cancelling training!')
#            self.model.stop_training = True

#callbacks = myCallback()

#Membuat Skema Model CNN

In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #Flaten
    tf.keras.layers.Flatten(),
    #batch normalization
    tf.keras.layers.BatchNormalization(),
    #dropout
    tf.keras.layers.Dropout(0.5, input_shape=(2,)),
    #fully connected
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')                                 
])

In [ ]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 148, 32)       896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 17, 128)      0

#Melatih Model CNN

In [ ]:
train_step_size = train_gen.n//train_gen.batch_size
val_step_size = val_gen.n//val_gen.batch_size

print(train_step_size)
print(val_step_size)

163
38


In [ ]:
model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',      
              metrics=['acc'])

In [ ]:
history = model2.fit_generator(train_gen, steps_per_epoch=87, epochs=100, validation_data= val_gen, validation_steps=10, verbose=2)  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/100


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


87/87 - 667s - loss: 0.7090 - acc: 0.8165 - val_loss: 0.6796 - val_acc: 0.7750 - 667s/epoch - 8s/step
Epoch 2/100
87/87 - 329s - loss: 0.3481 - acc: 0.8684 - val_loss: 0.6852 - val_acc: 0.5050 - 329s/epoch - 4s/step
Epoch 3/100
87/87 - 187s - loss: 0.3027 - acc: 0.8908 - val_loss: 0.6714 - val_acc: 0.7800 - 187s/epoch - 2s/step
Epoch 4/100
87/87 - 109s - loss: 0.2964 - acc: 0.8829 - val_loss: 0.5949 - val_acc: 0.6750 - 109s/epoch - 1s/step
Epoch 5/100
87/87 - 88s - loss: 0.2434 - acc: 0.9094 - val_loss: 0.5122 - val_acc: 0.7050 - 88s/epoch - 1s/step
Epoch 6/100
87/87 - 52s - loss: 0.2248 - acc: 0.9152 - val_loss: 0.2984 - val_acc: 0.8650 - 52s/epoch - 602ms/step
Epoch 7/100
87/87 - 36s - loss: 0.1999 - acc: 0.9293 - val_loss: 0.2248 - val_acc: 0.9050 - 36s/epoch - 413ms/step
Epoch 8/100
87/87 - 37s - loss: 0.2284 - acc: 0.9264 - val_loss: 0.2134 - val_acc: 0.9000 - 37s/epoch - 424ms/step
Epoch 9/100
87/87 - 32s - loss: 0.2152 - acc: 0.9279 - val_loss: 0.3184 - val_acc: 0.8400 - 32s/epo

#Simpan Model dan History

In [ ]:
import json

target_dir = '/content/drive/MyDrive/Machine Learning/Tubes/Model'

# simpan history model pertama
history_dict = history.history
json.dump(history_dict, open(target_dir + '/history_model2.json', 'w'))

# simpan model pertama dan weight-nya
model2.save(target_dir + '/model2.h5')
model2.save_weights(target_dir + '/weight_model2.hdf5')